<a href="https://colab.research.google.com/github/zhurbey/Surname_Generator/blob/master/Surname_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [0]:
#upload the data
from google.colab import files
uploaded = files.upload()

In [0]:
tf.test.is_gpu_available()

In [0]:
#creating alphabet
first_letter = ord('а')
last_letter = ord('я')
alphabet = []
for letter in range(first_letter,last_letter+1):
    alphabet.append(chr(letter))
    
#inserting extra symbols
alphabet.insert(6, "ё")
#inserting on empty space in order to be able to read shorter output
alphabet.insert(0, " ")
alphabet.insert(34,"-")
alphabet

In [0]:
#checking the lenth of the alphabet
len(alphabet)

In [0]:
#encoding a word function
def encode_word(word):
  #Downcase the word
  word = word.lower()
  x = 0
  encoded_word = []
  while x < 15:
    lenth = len(word)
    # add empty spaces to shorter words so imput is always 15 nodes
    if lenth < 15:
      difference = 15 - lenth
      while difference > 0:
        longer_word = word + " "
        word = longer_word
        difference = difference - 1
    # find index of a letter from the word    
    letter_index = alphabet.index(word[x])
    # add indeces to the list
    encoded_word.append(letter_index)
    x+=1
  return encoded_word

In [0]:
#normalizing encoded word function
def normalize_by(encoded_word):
  # normalised [0,1]
  normalized_word = (encoded_word - np.min([0]))/np.ptp([0,len(alphabet)])
  return normalized_word

In [50]:
#data to pandas DataFrame

df = pd.read_excel("Russian_Surname_250K.xlsx")
df.head()

,SURNAMES
0,ААЛФЕРОВА
1,ААЛЬ
2,ААМАН
3,ААМАНА
4,ААМАНАЯ


In [93]:
#For better perfomance we leave only surnames that are 8 letters long and end with "ОВ"(optional)
df['SURNAMES'] = df['SURNAMES'].astype('str')
mask = (df['SURNAMES'].str.len() == 8) & (df['SURNAMES'].str.endswith("ОВ"))
df.head()

,SURNAMES
13,АБАБИЛОВ
57,АБАКАРОВ
69,АБАКУЛОВ
73,АБАКУМОВ
88,АБАЛАКОВ


In [94]:
#while encoding encoder found there is a surname in data that has '\x1b' as a aprt of it, so this code is to remove that surname
#df[df['SURNAMES'].str.contains('\x1b')]
#df = df.drop(axis=0,index = 105298)

,SURNAMES
105298,КРАСНОВ


In [110]:
df.shape

(9158, 1)

In [0]:
#encode and normalize data
encoded_normalized_data = []
for word in df["SURNAMES"]:
  encoded_word = encode_word(word)
  normalized_word = normalize_by(encoded_word)
  encoded_normalized_data.append(normalized_word)
 

In [0]:
tf.reset_default_graph()

In [0]:
#creating generator function
def generator(z,reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        hidden1 = tf.layers.dense(inputs=z,units=128)
        # Leaky Relu
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1,hidden1)
        hidden2 = tf.layers.dense(inputs=hidden1,units=128)
        
        hidden2 = tf.maximum(alpha*hidden2,hidden2)
        output = tf.layers.dense(hidden2,units=15,activation=tf.nn.tanh)
        return output

In [0]:
#creating descriminator function
def discriminator(X,reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        hidden1 = tf.layers.dense(inputs=X,units=128)
        # Leaky Relu
        alpha = 0.01
        hidden1 = tf.maximum(alpha*hidden1,hidden1)
        
        hidden2 = tf.layers.dense(inputs=hidden1,units=128)
        hidden2 = tf.maximum(alpha*hidden2,hidden2)
        
        logits = tf.layers.dense(hidden2,units=1)
        output = tf.sigmoid(logits)
    
        return output, logits

In [0]:
#Creating placeholders
real_words = tf.placeholder(tf.float32,shape=[None,15])
z = tf.placeholder(tf.float32,shape=[None,15])

In [0]:
#Creating generator
G = generator(z)

In [0]:
#Creating discriminators
D_output_real , D_logits_real = discriminator(real_words)
D_output_fake, D_logits_fake = discriminator(G,reuse=True)


In [0]:
#Loss function
def loss_func(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

In [0]:
#Losses
D_real_loss = loss_func(D_logits_real,tf.ones_like(D_logits_real)* (0.9))

D_fake_loss = loss_func(D_logits_fake,tf.zeros_like(D_logits_real))

D_loss = D_real_loss + D_fake_loss

G_loss = loss_func(D_logits_fake,tf.ones_like(D_logits_fake))

In [0]:
learning_rate = 0.001

In [108]:
#Group variables
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

print([v.name for v in d_vars])
print([v.name for v in g_vars])

['dis/dense/kernel:0', 'dis/dense/bias:0', 'dis/dense_1/kernel:0', 'dis/dense_1/bias:0', 'dis/dense_2/kernel:0', 'dis/dense_2/bias:0']
['gen/dense/kernel:0', 'gen/dense/bias:0', 'gen/dense_1/kernel:0', 'gen/dense_1/bias:0', 'gen/dense_2/kernel:0', 'gen/dense_2/bias:0']


In [0]:
#Creating trainers
D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss, var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss, var_list=g_vars)

In [0]:
#Batch size and number of epochs
batch_size = 60
epochs = 500
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=g_vars)

In [0]:
# Next batch function
def next_batch(num, data):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
 
    #labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle) #np.asarray(labels_shuffle)

In [0]:
# Save a sample per epoch
samples = []

In [0]:
with tf.Session() as sess:
    
    sess.run(init)
    
    # Recall an epoch is an entire run through the training data
    for e in range(epochs):
        # // indicates classic division
        num_batches = len(newdf["SURNAMES"]) // batch_size
        
        for i in range(num_batches):
            
            # Grab batch of words
            batch = next_batch(batch_size, encoded_normalized_data)
            
            # Get words, reshape and rescale to pass to D
            batch_words = batch.reshape((batch_size, 15))
            
            
            
            # Z (random latent noise data for Generator)
            # -1 to 1 because of tanh activation
            batch_z = np.random.uniform(-1, 1, size=(batch_size, 15))
            
            # Run optimizers, no need to save outputs, we won't use them
            _ = sess.run(D_trainer, feed_dict={real_words: batch_words, z: batch_z})
            _ = sess.run(G_trainer, feed_dict={z: batch_z})
        
            
        print("Currently on Epoch {} of {} total...".format(e+1, epochs))
        
        # Sample from generator as we're training for viewing afterwards
        sample_z = np.random.uniform(-1, 1, size=(1, 15))
        gen_sample = sess.run(generator(z ,reuse=True),feed_dict={z: sample_z})
        
        samples.append(gen_sample)
        
#         saver.save(sess, './models/500_epoch_model.ckpt')

In [0]:
#denormalize function
def denormalize_by(encoded_word):
  
  normalized_word = (encoded_word - np.min([0]))*np.ptp([0,len(alphabet)])
  return normalized_word

In [0]:
#decode words function
def decode_words(output_words):
  decoded_words = []
  for word in output_words:
    
    decoded_word = []

    for x in word:
      x = np.round(x)
      
    decoded_word = x
    decoded_word = decoded_word.astype(int)
    decoded_word_string = ""
    a = 0
    
    while a < 15:
      
      b = decoded_word[a]
      
      if b < 0 or b > (len(alphabet)-1):
        b = 0
      b = alphabet[b]
      
      decoded_word_string = decoded_word_string + b

      a+=1

    decoded_words.append(decoded_word_string)


  return decoded_words





In [0]:
#denormalize Output
denorm_words = []
for word in samples:
  denormword = denormalize_by(word)
  denorm_words.append(denormword)

In [0]:
#Decode denormalized output
decoded_words = decode_words(denorm_words)

In [0]:
#See the result
decoded_words

In [0]:
#Save to Excel file and download
comletewords = pd.DataFrame(decoded_words)
file_name = 'OutputSurnames.xlsx'
writer = pd.ExcelWriter(file_name)
comletewords.to_excel(writer,'Sheet1')
writer.save()
downloaded = files.download(file_name)